In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import sklearn.metrics as sk_metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import tempfile
import statsmodels.api as sm
import os
from tensorflow import keras
import statsmodels.formula.api as smf

from tensorflow.keras import layers
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
import pymssql
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from tqdm import tqdm
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from mpl_toolkits import mplot3d


random_state = 2023

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

2.9.0


In [13]:
def get_column_index(cursor, column_name):
    for index, col in enumerate(cursor.description):
        if col[0] == column_name:
            return index
additional_data_headers = ["RentMortgagePayment" ,"TotalIncome" ,"RentMortgage" ,"CarPayment" ,"Expenses" ,"AvailableIncome" ,"Miles"]
additional_data_sql = "VIN,"
for i, header in enumerate(additional_data_headers):
    additional_data_sql += header
    if i!=len(additional_data_headers)-1:
        additional_data_sql+=','

conn = pymssql.connect(server='localhost', user='sa', password='Passw0rd!', database='Loans_2023_01_27')
cursor = conn.cursor()

headers = []
rows = []

# Underwriting Data
cursor.execute("SELECT * FROM dbo.UnderwritingData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "VIN")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rows.append(row)
    res = cursor.fetchone()

df = pd.DataFrame(rows, columns=headers)
ids = [i for i in range(0, len(df))]
default_status = ["NA" for _ in range(len(df))]
default_delq = ["NA" for i in range(len(df))]
default_apr = ["NA" for _ in range(len(df))]
df["ID"] = ids
df["APR"] = default_apr
df["Status"] = default_status
df["Delq"] = default_delq
df = df[df['OpeningBalance']!=0]


#Raw algorithm data
headers = []
rawAlgRows =[]
cursor.execute("Select * FROM dbo.RawAlgorithmData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "Vin6")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rawAlgRows.append(row)
    res = cursor.fetchone()

rawAlgDf = pd.DataFrame(rawAlgRows, columns=headers)
#prune extra space from end of vins
rawAlgDf['Vin6'] = rawAlgDf['Vin6'].apply(lambda x: x.strip())
df['VIN'] = df['VIN'].apply(lambda x: x.strip())


#matching by credit score
for idx, row in tqdm(rawAlgDf.iterrows()):
    vin = row["Vin6"]
    vinsFromUnder = df[df["VIN"]==vin]


    for index, match in vinsFromUnder.iterrows():
        matchID = match["ID"]
        underwritingCreditScore = (int)(match["CreaditScore"])
        rawCreditScore = (int)(row["Score"])

        #we don't want to match if the scores are zero, since that means they are missing
        if rawCreditScore==0:
            continue

        if underwritingCreditScore==rawCreditScore:
            df.loc[df["ID"]==matchID,"Status"]=row["ProcStatsCd"]
            df.loc[df["ID"]==matchID,"Delq"]=row["Delq"]
            df.loc[df["ID"]==matchID, "APR"] = int(row["Apr"] * 100) if row["Apr"] is not None else None #convert apr to an int

df = df[(df['Status']=='CO') | (df['Status']=='CL')]
df['Status'].replace(to_replace=['CO', 'CL'], value=[0,1], inplace=True)
df['HomeOwner'].replace(to_replace=['N', 'Y'], value=[0,1], inplace=True)
df['DelqPct'] = df.apply(lambda row : (100*row['Delq'])/row['OpeningBalance'],axis=1)
df.fillna(df.mean(), inplace=True)

#TODO maybe put back: CreaditScore, CurrentPaymentsMade, DelqPct
df = df[["OpeningBalance","Salary","Term","VehicleYear","LTV","DTI","TradelinesTotal","TradelinesBalance","Reposessions","Bankrupcies","Chageoff","Mortgage","NinetyDaysDelinquent","InquiresTotal","OpenTotal","ClosedTotalTermsNotPaid","LatePaymentHistory001230","LatePaymentHistory001260","LatePaymentHistory001290Plus","PaidAccounts","InquiriesLast6Months","OpenTotalInBadStatus","RentMortgage","CashDown","HomeOwner", "APR", "CreaditScore", "DelqPct", "Status"]]

#outlier handling
for col in df.columns:
    if(col in ["Status", "DelqPct", "CreaditScore", "HomeOwner", "Reposessions", "Chageoff", "APR"]):
        continue
    percentiles = df[col].quantile([0.00, 0.999]).values
    df.loc[df[col]<=percentiles[0], col] = percentiles[0]
    df.loc[df[col]>=percentiles[1], col] = percentiles[1]

26508it [00:25, 1026.51it/s]
/var/folders/kh/np0tdw052gg77h97rt6rm3l80000gn/T/ipykernel_77349/188036787.py:92: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [14]:
#0-1 Tranches for each feature
# for col in df.columns:
#     if col == "Status" or col =="DelqPct":
#         continue
#     # print(col+": ", end="")
#     # print(df[col].mean())
#     mean = df[col].mean()
#     df.loc[df[col]<=mean, col] = 0
#     df.loc[df[col]>mean, col] = 1

# print(df["DelqPct"].describe())
# td = df.copy(deep=True)
# td= td[td["DelqPct"]>0]
# print(td["DelqPct"].describe())

df.loc[(0 < df["DelqPct"]) & (df["DelqPct"] <= 4), "DelqPct"] = 1
df.loc[(4 < df["DelqPct"]) & (df["DelqPct"] <= 7), "DelqPct"] = 2
df.loc[(7 < df["DelqPct"]) & (df["DelqPct"] <= 11), "DelqPct"] = 3
df.loc[11 < df["DelqPct"], "DelqPct"] = 4

Xdat = df.iloc[:,:-2]
ydat = df["DelqPct"]


# print(Xdat)
count = [0,0,0,0,0]
print(ydat)
for i in ydat:
    count[int(i)]+=1
    # print(i)
print(count)

0        0.0
2        0.0
4        0.0
6        0.0
7        4.0
        ... 
16822    0.0
16832    0.0
16860    0.0
16887    0.0
17040    0.0
Name: DelqPct, Length: 8402, dtype: float64
[4401, 979, 809, 1077, 1136]


In [17]:
from scipy.stats import pearsonr

corrs = dict()

for col in df.columns:
    if col == "DelqPct" or col=="Status":
        continue
    corr, _ = pearsonr(df[col], df["DelqPct"])
    corrs[col] = abs(corr)

corrs = {k: v for k, v in sorted(corrs.items(), key=lambda item: item[1])}

goodCols = []

for col, score in corrs.items():
    if score>=0.1:
        goodCols.append(col)

print(goodCols)
goodCols.append("DelqPct")

df = df[goodCols]
Xdat = df.iloc[:,:-1]
ydat = df["DelqPct"]
print(Xdat)
print(ydat)

['Salary', 'NinetyDaysDelinquent', 'OpenTotalInBadStatus', 'OpeningBalance', 'CashDown', 'CreaditScore', 'APR']
       Salary  NinetyDaysDelinquent  OpenTotalInBadStatus  OpeningBalance  \
0      4604.0                     1                     1        13048.40   
4      4374.0                    20                    20        14975.10   
6      2800.0                     2                     2         8704.21   
7      1870.0                     5                     5         9395.40   
8      2500.0                     7                     7         9472.44   
...       ...                   ...                   ...             ...   
16720  3200.0                     0                     0        10487.67   
16805  1900.0                     2                     2        11042.58   
16822  2580.0                     5                     5         9111.68   
16860  5880.0                     0                     0         9119.37   
16887  2700.0                     3      

In [21]:
import random


for index, row in df.iterrows():
    if row["DelqPct"] == 0.0 and random.uniform(0, 1) < 0.25:
        df = df.drop(index)
    #     print("dropping")
    # else:
    #     print("not")


Xdat = df.iloc[:,:-1]
ydat = df["DelqPct"]
print(Xdat)
print(ydat)

count = [0,0,0,0,0]
print(ydat)
for i in ydat:
    count[int(i)]+=1
    # print(i)
print(count)

newydat = []
for i in ydat:
    if(i==0.0):
        newydat.append(0)
    else:
        newydat.append(1)

       Salary  NinetyDaysDelinquent  OpenTotalInBadStatus  OpeningBalance  \
4      4374.0                    20                    20        14975.10   
7      1870.0                     5                     5         9395.40   
8      2500.0                     7                     7         9472.44   
11     2500.0                     1                     1         8557.73   
12     1569.0                     2                     2        10843.18   
...       ...                   ...                   ...             ...   
16592  4700.0                     0                     0        17954.18   
16633  4411.0                     5                     5        16250.00   
16710  2700.0                     0                     0        11500.00   
16805  1900.0                     2                     2        11042.58   
16860  5880.0                     0                     0         9119.37   

       CashDown  CreaditScore   APR  
4           0.0           485  2499  

In [22]:
accs = []
for x in tqdm(range(3)):
    x_train, x_test, y_train, y_test = train_test_split(Xdat, newydat, test_size=0.2)
    train = xgb.DMatrix(x_train, label=y_train, enable_categorical=True)
    test = xgb.DMatrix(x_test, label=y_test, enable_categorical=True)

    param = {
        'eta': 0.01,
        'objective': 'multi:softprob',
        'num_class': 2
    }
    epochs = 500

    model = xgb.train(param, train, epochs)

    predictions = model.predict(test)

    y = []

    for z in predictions:
        y.append(np.argmax(z))

    acc = round(accuracy_score(y_test, y), 3)
    # model.save_model('Models/Delq/XGBoost_{}%_Scorelate.json'.format(acc))

    print("Acc: ", end='')
    print(acc)

    accs.append(acc)

    pred = model.predict(xgb.DMatrix(x_test))
    rpreds = [1 if i[0]>=0.5 else 0 for i in pred]
    print("Preds")
    print(pred[:10])
    print("Test")
    print(y_test[:10])

print(sum(accs)/len(accs))

 33%|███▎      | 1/3 [00:05<00:11,  5.76s/it]

Acc: 0.741
Preds
[[0.27317795 0.7268221 ]
 [0.18525247 0.8147475 ]
 [0.20330165 0.79669833]
 [0.18620454 0.8137955 ]
 [0.31254125 0.68745875]
 [0.22742315 0.77257687]
 [0.22601572 0.77398425]
 [0.19010635 0.80989367]
 [0.7235189  0.27648106]
 [0.29948112 0.70051885]]
Test
[0, 1, 1, 0, 1, 0, 1, 1, 0, 1]


 67%|██████▋   | 2/3 [00:12<00:06,  6.07s/it]

Acc: 0.758
Preds
[[0.18378083 0.8162192 ]
 [0.24424413 0.7557559 ]
 [0.21189184 0.7881082 ]
 [0.26854497 0.73145497]
 [0.22507647 0.77492356]
 [0.29109216 0.70890784]
 [0.17091331 0.8290867 ]
 [0.22009386 0.77990615]
 [0.30592763 0.69407237]
 [0.20368242 0.7963175 ]]
Test
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]


 67%|██████▋   | 2/3 [00:14<00:07,  7.29s/it]


KeyboardInterrupt: 